In [1]:
from lemat_genbench.benchmarks.validity_benchmark import ValidityBenchmark
from typing import Any, Dict, List

from lemat_genbench.preprocess.validity_preprocess import ValidityPreprocessor
from lemat_genbench.utils.logging import logger
from pathlib import Path
from tqdm import tqdm
from datasets import load_dataset
import numpy as np
import pandas as pd
from pymatgen.core import Structure
import time 
import gc
import torch

Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
W0923 23:04:46.154000 53888 torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\samue\lemat-genbench\.venv\Lib\site-packages\material_hasher\similarity\__init__.py", line 13, in <module>
    from .eqv2 import EquiformerV2Similarity
  File "C:\Users\samue\lemat-genbench\.venv\Lib\site-packages\material_hasher\similarity\eqv2.py", line 12, in <module>
    from fairchem.core import OCPCalculator
ImportError: cannot import name 'OCPCalculator' from 'fairchem.core' (C:\Users\samue\lemat-genbench\.venv\Lib\site-packages\fairchem\core\__init__.py)

During handling of the above exception, another exception occurred:

Traceback (most recent cal

In [2]:
def lematbulk_item_to_structure(item: dict):
    sites = item["species_at_sites"]
    coords = item["cartesian_site_positions"]
    cell = item["lattice_vectors"]

    structure = Structure(
        species=sites, coords=coords, lattice=cell, coords_are_cartesian=True
    )

    return structure

In [3]:
def run_validity_preprocessing_and_filtering(
    structures, config: Dict[str, Any], monitor_memory: bool = False
):
    """Run validity benchmark and preprocessing, then filter to valid structures only.

    Returns
    -------
    tuple
        (validity_benchmark_result, valid_structures, validity_filtering_metadata)
    """
    # Log initial memory usage

    n_total_structures = len(structures)
    logger.info(
        f"🔍 Starting MANDATORY validity processing for {n_total_structures} structures..."
    )

    # Step 1: Run validity benchmark on ALL structures
    logger.info("🔍 Running MANDATORY validity benchmark on ALL structures...")
    start_time = time.time()

    validity_settings = config.get("validity_settings", {})
    validity_benchmark = ValidityBenchmark(
        charge_tolerance=validity_settings.get("charge_tolerance", 0.1),
        distance_scaling=validity_settings.get("distance_scaling", 0.5),
        min_density=validity_settings.get("min_density", 0.01),
        max_density=validity_settings.get("max_density", 25.0),
        check_format=validity_settings.get("check_format", True),
        check_symmetry=validity_settings.get("check_symmetry", True),
    )

    validity_benchmark_result = validity_benchmark.evaluate(structures)

    elapsed_time = time.time() - start_time
    logger.info(
        f"✅ MANDATORY validity benchmark complete for {n_total_structures} structures in {elapsed_time:.1f}s"
    )

    # Clean up after validity benchmark
    cleanup_after_benchmark("validity", monitor_memory)

    # Step 2: Run validity preprocessor on ALL structures
    logger.info("🔍 Running MANDATORY validity preprocessor on ALL structures...")
    start_time = time.time()

    charge_tolerance = validity_settings.get("charge_tolerance", 0.1)
    distance_scaling = validity_settings.get("distance_scaling", 0.5)
    min_density = validity_settings.get("min_density", 0.01)
    max_density = validity_settings.get("max_density", 25.0)
    check_format = validity_settings.get("check_format", True)
    check_symmetry = validity_settings.get("check_symmetry", True)

    validity_preprocessor = ValidityPreprocessor(
        charge_tolerance=charge_tolerance,
        distance_scaling_factor=distance_scaling,
        plausibility_min_density=min_density,
        plausibility_max_density=max_density,
        plausibility_check_format=check_format,
        plausibility_check_symmetry=check_symmetry,
    )

    # Create source IDs for tracking
    structure_sources = [f"structure_{i}" for i in range(len(structures))]
    validity_preprocessor_result = validity_preprocessor.run(
        structures, structure_sources=structure_sources
    )
    processed_structures = validity_preprocessor_result.processed_structures

    elapsed_time = time.time() - start_time
    logger.info(
        f"✅ MANDATORY validity preprocessing complete for {len(processed_structures)} structures in {elapsed_time:.1f}s"
    )

    # Clean up after validity preprocessor
    cleanup_after_preprocessor("validity", monitor_memory)

    # Step 3: Filter to only valid structures
    logger.info("🔍 Filtering to valid structures only...")

    valid_structures = []
    valid_structure_ids = []
    valid_structure_sources = []

    for structure in processed_structures:
        is_valid = structure.properties.get("overall_valid", False)
        if is_valid:
            valid_structures.append(structure)
            valid_structure_ids.append(
                structure.properties.get("structure_id", "unknown")
            )
            valid_structure_sources.append(
                structure.properties.get("original_source", "unknown")
            )

    n_valid_structures = len(valid_structures)
    n_invalid_structures = n_total_structures - n_valid_structures

    # Log filtering results
    logger.info(
        f"✅ Filtering complete: {n_valid_structures} valid structures out of {n_total_structures} total"
    )
    logger.info(f"📊 Valid: {n_valid_structures}, Invalid: {n_invalid_structures}")

    if n_valid_structures == 0:
        logger.warning(
            "⚠️  No valid structures found! All subsequent benchmarks will be skipped."
        )

    # Create filtering metadata
    validity_filtering_metadata = {
        "total_input_structures": n_total_structures,
        "valid_structures": n_valid_structures,
        "invalid_structures": n_invalid_structures,
        "validity_rate": n_valid_structures / n_total_structures
        if n_total_structures > 0
        else 0.0,
        "valid_structure_ids": valid_structure_ids,
        "valid_structure_sources": valid_structure_sources,
    }

    # Log final memory usage
    # log_memory_usage("after validity filtering", force_log=monitor_memory)

    return validity_benchmark_result, valid_structures, validity_filtering_metadata, processed_structures

In [4]:
def cleanup_after_benchmark(benchmark_name: str, monitor_memory: bool = False):
    """Clean up memory after running a benchmark."""
    logger.info(f"🧹 Cleaning up after {benchmark_name} benchmark...")

    # Clear memory
    clear_memory()

    # Log memory usage
    # log_memory_usage(f"after {benchmark_name} cleanup", force_log=monitor_memory)

In [5]:
def clear_memory():
    """Clear memory by running garbage collection and clearing PyTorch cache."""
    # Run Python garbage collection
    gc.collect()

    # Clear PyTorch cache if available
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Force garbage collection again
    gc.collect()

    logger.debug("🧹 Memory cleared (garbage collection + PyTorch cache)")

In [6]:
def cleanup_after_preprocessor(preprocessor_name: str, monitor_memory: bool = False):
    """Clean up memory after running a preprocessor."""
    logger.info(f"🧹 Cleaning up after {preprocessor_name} preprocessor...")

    # Clear memory
    clear_memory()

    # Clear MLIP models if it was a MLIP preprocessor
    if "mlip" in preprocessor_name.lower():
        clear_mlip_models()

    # Log memory usage
    # log_memory_usage(f"after {preprocessor_name} cleanup", force_log=monitor_memory)

In [7]:
dataset_name = "Lematerial/LeMat-Bulk"
name = "compatible_pbe"
split = "train"
dataset = load_dataset(dataset_name, name=name, split=split, streaming=False)

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [20]:
np.random.seed(32)
indicies = np.random.randint(0, len(dataset), 6000)

In [31]:
indicies[5]

np.int32(384403)

In [32]:
dataset[384403]

{'elements': ['Ho', 'In', 'N', 'Pm'],
 'nsites': 5,
 'chemical_formula_anonymous': 'A2BCD',
 'chemical_formula_reduced': 'Ho2InNPm',
 'chemical_formula_descriptive': 'Pm1 Ho2 In1 N1',
 'nelements': 4,
 'dimension_types': [1, 1, 1],
 'nperiodic_dimensions': 3,
 'lattice_vectors': [[4.97859257, 0.0, 0.0],
  [0.0, 4.80446104, 0.0],
  [0.0, 0.0, 4.80446104]],
 'immutable_id': 'agm005014102',
 'cartesian_site_positions': [[2.489296285, 0.0, 0.0],
  [0.0, 0.0, 2.40223052],
  [0.0, 2.40223052, 0.0],
  [2.489296285, 2.40223052, 2.40223052],
  [0.0, 0.0, 0.0]],
 'species': '[{"mass": null, "name": "In", "attached": null, "nattached": null, "concentration": [1], "original_name": null, "chemical_symbols": ["In"]}, {"mass": null, "name": "Ho", "attached": null, "nattached": null, "concentration": [1], "original_name": null, "chemical_symbols": ["Ho"]}, {"mass": null, "name": "Pm", "attached": null, "nattached": null, "concentration": [1], "original_name": null, "chemical_symbols": ["Pm"]}, {"mass"

In [21]:
# row = []
structures = []
for i in tqdm(range(len(indicies))):
    # print(index)
    index = int(indicies[i])
    strut = lematbulk_item_to_structure(dataset[index])
    structures.append(strut)
    # name = dataset[index]["immutable_id"]
    # row.append([name, strut])

100%|███████████████████████████████████████████████████████████████████████████| 6000/6000 [00:05<00:00, 1147.37it/s]


In [22]:
# lematbulk_df = pd.DataFrame(row, columns = ["immutable_id", "LeMatStructs"])
# lematbulk_df.to_csv("lematbulk_sample_df.csv")

In [23]:
# lematbulk_df = pd.read_csv("lematbulk_sample_df.csv")

In [24]:
config = {
    "validity": True,  # ALWAYS run validity preprocessing
    "distribution": False,
    "stability": False,
    "embeddings": False,
    "fingerprint": False,
}

In [25]:
result = run_validity_preprocessing_and_filtering(structures, config=config)

[23:12:15] INFO     🔍 Starting MANDATORY validity processing for 6000 structures...               ]8;id=917245;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py\2631243110.py]8;;\:]8;id=306863;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py#14\14]8;;\

           INFO     🔍 Running MANDATORY validity benchmark on ALL structures...                   ]8;id=347133;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py\2631243110.py]8;;\:]8;id=457530;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py#19\19]8;;\

IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError
IndexError

[23:18:35] INFO     ✅ MANDATORY validity benchmark complete for 6000 structures in 380.0s         ]8;id=309934;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py\2631243110.py]8;;\:]8;id=581052;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py#35\35]8;;\

           INFO     🧹 Cleaning up after validity benchmark...                                       ]8;id=96221;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\307847465.py\307847465.py]8;;\:]8;id=766323;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\307847465.py#3\3]8;;\

...

[23:21:45] INFO     ✅ MANDATORY validity preprocessing complete for 6000 structures in 189.6s     ]8;id=960399;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py\2631243110.py]8;;\:]8;id=56717;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py#70\70]8;;\

           INFO     🧹 Cleaning up after validity preprocessor...                                   ]8;id=91260;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\3181986820.py\3181986820.py]8;;\:]8;id=664528;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\3181986820.py#3\3]8;;\

[23:21:46] INFO     🔍 Filtering to valid structures only...                                       ]8;id=715152;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py\2631243110.py]8;;\:]8;id=92464;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py#78\78]8;;\

           INFO     ✅ Filtering complete: 5131 valid structures out of 6000 total                 ]8;id=680238;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py\2631243110.py]8;;\:]8;id=184489;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py#99\99]8;;\

           INFO     📊 Valid: 5131, Invalid: 869                                                  ]8;id=937136;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py\2631243110.py]8;;\:]8;id=709645;file://C:\Users\samue\AppData\Local\Temp\ipykernel_53888\2631243110.py#102\102]8;;\

In [26]:
validity_benchmark_result, valid_structures, validity_filtering_metadata, processed_structures = result

In [27]:
invalid_indicies = []
count = 0
for val in validity_benchmark_result.evaluator_results["charge_neutrality"]["metric_results"]["charge_neutrality"].individual_values:
    if val == 10.0:
        invalid_indicies.append(count)
    count += 1


In [14]:
invalid_indicies

[5, 7, 13, 18, 23, 35, 45, 46]

In [19]:
processed_structures[13]

Structure Summary
Lattice
    abc : 5.477279534771641 5.477279795045079 9.449985872135557
 angles : 73.15380538709327 73.1538046207681 60.00000107406842
 volume : 231.36765456958355
      A : np.float64(5.25116589) np.float64(0.0) np.float64(1.55751337)
      B : np.float64(2.39460115) np.float64(4.67339613) np.float64(1.55751337)
      C : np.float64(0.05372929) np.float64(0.03284148) np.float64(9.44977606)
    pbc : True True True
PeriodicSite: Pm (4.72, 2.885, 3.348) [0.6163, 0.6163, 0.1512]
PeriodicSite: Pm (0.115, 0.07031, 9.243) [0.00819, 0.00819, 0.9754]
PeriodicSite: Pm (3.734, 2.282, 6.674) [0.4845, 0.4845, 0.5465]
PeriodicSite: H (1.486, 0.9085, 4.62) [0.1914, 0.1914, 0.4258]
PeriodicSite: Pt (3.095, 0.2643, 8.828) [0.5577, 0.05069, 0.8339]
PeriodicSite: Pt (1.651, 2.656, 4.066) [0.05291, 0.5659, 0.3283]
PeriodicSite: Pt (4.309, 2.634, 9.618) [0.5577, 0.5577, 0.8339]
PeriodicSite: Pt (0.4222, 0.2581, 3.267) [0.05291, 0.05291, 0.3283]
PeriodicSite: Pt (6.189, 3.783, 8.027) [0.

In [16]:
processed_structures[5].properties

{'_validity_diagnostics': {'overall_valid': 0.0,
  'charge_valid': False,
  'distance_valid': True,
  'plausibility_valid': True},
 'structure_id': 5,
 'original_source': 'structure_5',
 'overall_valid': False,
 'charge_valid': False,
 'distance_valid': True,
 'plausibility_valid': True,
 'charge_deviation': 10.0,
 'distance_score': 1.0,
 'plausibility_score': 1.0,
 'validity_details': {'charge_neutrality': {'valid': False,
   'deviation': 10.0,
   'tolerance': 0.1},
  'interatomic_distance': {'valid': True, 'score': 1.0},
  'physical_plausibility': {'valid': True, 'score': 1.0},
  'overall': {'valid': False, 'checks_passed': 2, 'total_checks': 3}},
 'validity_preprocessor_version': '1.1.0',
 'validity_timestamp': '1758661496'}

In [15]:
invalid_indicies

[3, 5, 7, 13, 18, 23, 24, 25, 35, 45, 46, 52, 57]

In [116]:
print([processed_structures[i].composition for i in invalid_indicies])

[Composition('Ca3 Th1 Pa3 N8'), Composition('Pm1 Ho2 In1 N1'), Composition('As1 Cu2 Pd1'), Composition('Pm3 H1 Pt8'), Composition('Ac2 Sn2 H4 Ru1'), Composition('Sm4 Pa2 Se6'), Composition('B4 Hf4 Tc2'), Composition('Au1 Al1 Se2'), Composition('Hg2 Se1 Ta1'), Composition('K2 Er4 S3 O2'), Composition('Na4 Se2 Ta4'), Composition('Cs4 Pt1 Te1'), Composition('Mo1 Os1 P2'), Composition('Hf1 Tl1 In2 Br8'), Composition('Pb1 Au1 N2'), Composition('Cr1 Sb1 Zn1'), Composition('Mo2 Fe1 Cl1'), Composition('S1 Hg3 Ta1'), Composition('Pr2 Sm1 Np1 Sb4'), Composition('Pa10 Si4 Te4'), Composition('Au4 I4 Te4'), Composition('Si2 Cu2 Ce1'), Composition('K3 Tc1 Rh3 O8'), Composition('I2 Sc1 Ba1'), Composition('Os2 S2 Tb2'), Composition('Tl2 S6 Pt2 Er2'), Composition('H1 Hf1 Sn1 V2'), Composition('Hg4 P1 Os1'), Composition('Mn1 Si1 Sr2'), Composition('H4 Ba4 Cu2'), Composition('As1 Au1 Co1 Tc1'), Composition('Rb4 Ac2 Tl2 S8'), Composition('Ni1 Tc2 W1 H1'), Composition('Sr2 Hg2 Br1'), Composition('Co1 Te1 P

In [28]:
validity_filtering_metadata

{'total_input_structures': 6000,
 'valid_structures': 5131,
 'invalid_structures': 869,
 'validity_rate': 0.8551666666666666,
 'valid_structure_ids': [0,
  1,
  2,
  3,
  4,
  6,
  8,
  9,
  10,
  11,
  12,
  14,
  15,
  16,
  17,
  19,
  20,
  21,
  22,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  71,
  72,
  73,
  74,
  75,
  76,
  79,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  90,
  91,
  92,
  93,
  94,
  95,
  98,
  99,
  100,
  101,
  102,
  103,
  105,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,

In [65]:
# Imports
from pymatgen.core import Structure
from pymatgen.analysis.bond_valence import calculate_bv_sum

def bvs_calculator(structure):
    # Loop through all sites in structure
    bv = []
    for site in structure.sites:
        # Get neighbors of site
        nn_list = structure.get_neighbors(site, 4)
        # Calculate bond valence sum of site
        bvs_value = calculate_bv_sum(site, nn_list)
        # Print site specie and bond valence sum
        bv.append(bvs_value)
        # print(site.specie, bvs_value)
    # print(bv)
    # print("total = " + str(sum(bv)))
    # print(" ")

In [67]:
test_inds = []
for ind in range(0,6000):
    # print(ind)
    try:
        bvs_calculator(processed_structures[ind])
    except KeyError:
        # print(" ")
        test_inds.append(ind) 

In [68]:
print(len(test_inds))

618


In [62]:
bvs_calculator(processed_structures[435])

Ru 0.03925888490038114
Ru 0.03925888490038116


KeyError: Element Tc

In [40]:
test_inds

[5,
 13,
 18,
 23,
 80,
 97,
 134,
 220,
 273,
 320,
 322,
 341,
 387,
 435,
 440,
 475,
 517,
 519,
 526,
 618,
 655,
 742,
 763,
 772,
 850,
 871,
 898,
 955,
 960,
 966,
 978,
 983,
 1150,
 1184,
 1187,
 1231,
 1233,
 1274,
 1284,
 1338,
 1370,
 1377,
 1458,
 1542,
 1578,
 1582,
 1590,
 1618,
 1623,
 1633,
 1809,
 1833,
 1850,
 1865,
 1872,
 1925,
 1980,
 2018,
 2073,
 2087,
 2106,
 2119,
 2139,
 2195,
 2210,
 2261,
 2316,
 2343,
 2368,
 2429,
 2649,
 2655,
 2681,
 2686,
 2701,
 2735,
 2749,
 2756,
 2775,
 2796,
 2834,
 2919,
 3021,
 3045,
 3048,
 3112,
 3120,
 3138,
 3153,
 3179,
 3185,
 3246,
 3256,
 3281,
 3284,
 3347,
 3374,
 3409,
 3465,
 3477,
 3545,
 3551,
 3661,
 3675,
 3680,
 3710,
 3778,
 3779,
 3976,
 4011,
 4034,
 4109,
 4110,
 4140,
 4195,
 4199,
 4216,
 4231,
 4239,
 4242,
 4250,
 4269,
 4315,
 4350,
 4352,
 4419,
 4443,
 4458,
 4481,
 4552,
 4611,
 4615,
 4625,
 4661,
 4730,
 4743,
 4747,
 4748,
 4761,
 4762,
 4884,
 4915,
 4926,
 4952,
 4981,
 4997,
 5055,
 5112,
 51

In [41]:
invalid_indicies

[5,
 7,
 13,
 18,
 23,
 35,
 45,
 46,
 70,
 77,
 78,
 80,
 89,
 96,
 97,
 104,
 106,
 134,
 145,
 160,
 161,
 166,
 169,
 171,
 174,
 180,
 182,
 188,
 191,
 194,
 195,
 203,
 211,
 220,
 222,
 223,
 228,
 238,
 242,
 251,
 255,
 263,
 273,
 276,
 277,
 278,
 280,
 288,
 291,
 292,
 293,
 306,
 307,
 311,
 320,
 322,
 326,
 341,
 345,
 364,
 374,
 381,
 383,
 387,
 389,
 390,
 401,
 402,
 404,
 435,
 439,
 440,
 447,
 461,
 466,
 473,
 474,
 475,
 489,
 495,
 507,
 510,
 517,
 519,
 521,
 526,
 534,
 541,
 550,
 580,
 581,
 583,
 585,
 586,
 590,
 591,
 609,
 614,
 618,
 624,
 627,
 631,
 638,
 655,
 659,
 661,
 662,
 675,
 676,
 685,
 695,
 700,
 730,
 742,
 755,
 763,
 772,
 777,
 789,
 791,
 823,
 840,
 848,
 850,
 858,
 871,
 875,
 887,
 888,
 898,
 899,
 904,
 915,
 917,
 922,
 924,
 937,
 941,
 943,
 949,
 955,
 960,
 966,
 978,
 981,
 983,
 993,
 994,
 1016,
 1020,
 1021,
 1026,
 1031,
 1034,
 1044,
 1056,
 1062,
 1076,
 1079,
 1084,
 1087,
 1093,
 1097,
 1105,
 1111,
 1117,
 11